In [17]:
url_bookings = '../data/Bookings.csv'
url_prop_scrap = '../data/properties_scrapping.csv'
url_prop = '../data/properties.csv'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Diccionario de datos de `Booking.csv`

- **PropertyId**: El ID de la propiedad. 

- **Property_BookingId**: El ID de la reserva para esa propiedad. 

- **BookingCreatedDate**: La fecha y hora en que se creó la reserva. 

- **ArrivalDate**: La fecha de llegada o check-in, cuando el huésped llegará a la propiedad. 

- **DepartureDate**: La fecha de salida o check-out, cuando el huésped dejará la propiedad. 

- **Adults**: El número de adultos en la reserva. 

- **Children**: El número de niños en la reserva. 

- **Infants**: El número de bebés (infantes) en la reserva.

- **Persons**: El número total de personas en la reserva, que es la suma de adultos, niños y bebés. 

- **NumNights**: El número total de noches que cubre la reserva. 

- **Channel**: El canal de reserva o la plataforma utilizada para hacer la reserva.

- **RoomRate**: La tarifa de la habitación por el total de la estancia (antes de otras tarifas). 

- **CleaningFee**: La tarifa de limpieza asociada con la reserva. 

- **Revenue**: El total de ingresos por la reserva antes de impuestos

- **ADR**: "Average Daily Rate" o Tarifa Media Diaria, que es la tarifa promedio por noche para la reserva. 

- **TouristTax**: El impuesto turístico asociado a la reserva.

- **TotalPaid**: El monto total pagado por el huésped, que incluye tarifas de la habitación y cualquier otro cargo o impuesto. 

### ETL `Booking.csv`

In [143]:
df_bookings = pd.read_csv(url_bookings)

rename_booking = {
    'Property_BookingId': 'booking_id',
    'BookingCreatedDate': 'created_date',
    'ArrivalDate': 'check_in_date',
    'DepartureDate': 'check_out_date',
    'NumNights': 'n_nights',
    'Adults': 'n_adults',
    'Children': 'n_children',
    'Infants': 'n_infants',
    'RoomRate': 'total_without_extras',
    'ADR': 'avg_rate_night',
    'Channel': 'channel',
    'TotalPaid': 'total_paid',
    'PropertyId': 'property_id'
}

columns_booking = list(rename_booking.keys())
dim_booking = df_bookings[columns_booking].rename(columns=rename_booking)
cond_fmt_one = dim_booking['created_date'].str.contains(r'\d{2}/\d{2}/\d{4}', regex=True)
dim_booking.loc[cond_fmt_one, 'created_date'] = pd.to_datetime(dim_booking[cond_fmt_one]['created_date'], format='%d/%m/%Y')

dim_booking.loc[dim_booking['n_nights'] <= 0, 'n_nights'] = np.nan
dim_booking.loc[dim_booking['avg_rate_night'] == 0, 'avg_rate_night'] = np.nan
dim_booking.loc[dim_booking['total_without_extras'] == 0, 'total_without_extras'] = np.nan
dim_booking.loc[dim_booking['channel'].isna(), 'channel'] = 'Otros'
dim_booking['channel'] = dim_booking['channel'].str.replace(r'\..*', '', regex=True)
dim_booking.loc[dim_booking['total_paid'] == 0, 'total_paid'] = np.nan

dim_booking.loc[dim_booking['total_without_extras'] < 0, 'total_without_extras'] = dim_booking[dim_booking['total_without_extras'] < 0].\
    apply(lambda row: 
          np.nan if np.abs(row['total_without_extras']) < row['avg_rate_night'] else row['total_without_extras'] * -1, axis=1
          )

dim_booking.loc[dim_booking['total_paid'] < 0, 'total_paid'] = dim_booking[dim_booking['total_paid'] < 0].\
    apply(lambda row: 
          np.nan if row['total_without_extras'] > np.abs(row['total_paid']) else row['total_paid'] * -1, axis=1
          )

dim_booking = dim_booking.astype({'n_adults': 'int', 'n_children': 'int', 'n_infants': 'int'})
dim_booking['channel'] = dim_booking['channel'].str.lower()

In [144]:
dim_booking

,booking_id,created_date,check_in_date,check_out_date,n_nights,n_adults,n_children,n_infants,total_without_extras,avg_rate_night,channel,total_paid,property_id
0,155168,2024-10-03 16:42:13,2024-10-09 00:00:00,2024-10-12 00:00:00,3.0,2,0,0,391.03,130.340000,airbnb,394.99,43469
1,155167,2024-10-03 00:00:00,2025-02-02 00:00:00,2025-02-07 00:00:00,5.0,3,0,0,1692.00,358.630000,booking,1808.13,43025
2,155166,2024-10-03 00:00:00,2024-11-18 00:00:00,2024-11-25 00:00:00,7.0,3,0,0,827.17,118.170000,airbnb,971.55,43404
3,155165,2024-10-03 15:55:39,2024-11-14 00:00:00,2024-11-18 00:00:00,4.0,5,0,0,692.86,173.220000,airbnb,830.36,43276
4,155164,2024-10-03 15:53:02,2024-11-20 00:00:00,2024-12-06 00:00:00,16.0,5,0,0,2005.43,125.340000,airbnb,2246.06,4138
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79590,48909,2012-01-16 22:09:11,2012-05-25 00:00:00,2012-05-28 11:00:00,3.0,6,0,0,778.39,259.463333,rentthesun,NaN,2883
79591,48887,2012-01-12 00:00:00,2012-02-04 00:00:00,2012-02-08 11:00:00,3.0,4,0,1,576.21,144.052500,rentthesun,NaN,2883
79592,48865,2012-01-08 17:05:53,2012-04-06 00:00:00,2012-04-13 11:00:00,7.0,6,0,0,1461.76,208.822857,rentthesun,NaN,2883
79593,48819,2011-12-22 18:18:12,2011-12-28 00:00:00,2012-01-06 11:00:00,9.0,2,0,0,1000.00,111.111111,rentthesun,NaN,2883


## Diccionario de datos de `Properties.csv`

- **PropertyId**: El ID de la propiedad. Es un identificador único que se utiliza para referirse a esta propiedad específica.

- **RealProperty**: Indica si la propiedad es real o disponible para reserva o venta. En este caso, Yes indica que la propiedad está disponible.

- **Capacity**: La capacidad máxima de la propiedad en términos del número de personas que puede alojar. 

- **Square**: El tamaño de la propiedad en metros cuadrados. En este caso, la propiedad tiene un tamaño de 141 metros cuadrados.

- **PropertyType**: El tipo de propiedad. 

- **NumBedrooms**: El número de habitaciones en la propiedad. 

- **ReadyDate**: La fecha en la que la propiedad estará lista o disponible para ser ocupada, probablemente después de una construcción, remodelación o preparación.

In [131]:
df_properties = pd.read_csv(url_prop)
df_properties

,PropertyId,RealProperty,Capacity,Square,PropertyType,NumBedrooms,ReadyDate
0,43630,Yes,7,141,Apartment,3,2024-09-25 00:00:00
1,43622,Yes,4,70,Apartment,2,2024-08-12 00:00:00
2,43620,Yes,2,55,Apartment,1,2024-08-04 00:00:00
3,43616,No,4,60,Apartment,2,2024-07-05 00:00:00
4,43606,Yes,4,60,Apartment,2,2024-06-27 00:00:00
...,...,...,...,...,...,...,...
339,43069,Yes,10,110,Apartment,3,2020-02-01 00:00:00
340,9973,Yes,7,60,Apa,2,2016-02-14 00:00:00
341,43056,Yes,5,50,Apartment,1,2019-10-25 00:00:00
342,21699,Yes,5,69,Apa,2,2016-05-25 00:00:00


### ETL `Properties.csv`

Para casos donde tenga valores nulos, reemplazaremos por una categoría de `otros`

In [140]:
rename_properties = {
    'PropertyId': 'property_id',
    'Capacity': 'capacity',
    'Square': 'square_mts',
    'PropertyType': 'property_type',
    'NumBedrooms': 'n_bedrooms',
}

mapping_type_prop = {
    'Apa': 'apartamento',
    'Apartment': 'apartamento',
    'House': 'casa'
}

df_properties = pd.read_csv(url_prop)

columns_properties = list(rename_properties.keys())
dim_properties_owner = df_properties[columns_properties].rename(columns=rename_properties)
dim_properties_owner.loc[dim_properties_owner['property_type'].isna(), 'property_type'] = 'otros'
dim_properties_owner.drop_duplicates(subset=['property_id'], inplace=True)
dim_properties_owner['property_type'] = dim_properties_owner['property_type'].replace(mapping_type_prop)
dim_properties_owner

,property_id,capacity,square_mts,property_type,n_bedrooms
0,43630,7,141,apartamento,3
1,43622,4,70,apartamento,2
2,43620,2,55,apartamento,1
3,43616,4,60,apartamento,2
4,43606,4,60,apartamento,2
...,...,...,...,...,...
339,43069,10,110,apartamento,3
340,9973,7,60,apartamento,2
341,43056,5,50,apartamento,1
342,21699,5,69,apartamento,2


# Diccionario de datos de `properties_scrapping.csv`


- **property_id**: Identificador único de la propiedad.

- **property_name**: Nombre o descripción de la propiedad.

- **reference_rate_night**: Tarifa de referencia por noche de la propiedad.

- **rating**: Puntuación promedio basada en las reseñas de los huéspedes (de 1 a 5).

- **n_reviews**: Número de reseñas que tiene la propiedad.

- **city**: Ciudad donde está ubicada la propiedad.

- **country**: País donde se encuentra la propiedad.

- **property_type**: Tipo de propiedad (e.g. habitación, apartamento, etc.).

- **url_property**: Enlace a la página web de la propiedad en Airbnb.

### ETL `properties_scrapping.csv`


In [142]:
dim_properties_competitor = pd.read_csv(url_prop_scrap)
dim_properties_competitor[['property_name', 'city', 'country', 'property_type']] = dim_properties_competitor[['property_name', 'city', 'country', 'property_type']].apply(lambda x: x.str.lower())
dim_properties_competitor.drop_duplicates(subset=['property_id'], inplace=True)
dim_properties_competitor

,property_id,property_name,reference_rate_night,rating,n_reviews,city,country,property_type,url_property
0,1221751561372481946,habitación en barcelona,29.0,4.72,18,barcelona,españa,habitación,https://www.airbnb.com.pe/rooms/12217515613724...
1,1218645303173234166,habitación en barcelona,46.0,4.93,15,barcelona,españa,habitación,https://www.airbnb.com.pe/rooms/12186453031732...
2,27127886,habitación en barcelona,36.0,4.92,284,barcelona,españa,habitación,https://www.airbnb.com.pe/rooms/27127886?adult...
3,41822352,habitación en barcelona,46.0,4.91,23,barcelona,españa,habitación,https://www.airbnb.com.pe/rooms/41822352?adult...
4,6833040,habitación en barcelona,37.0,4.52,292,barcelona,españa,habitación,https://www.airbnb.com.pe/rooms/6833040?adults...
...,...,...,...,...,...,...,...,...,...
265,994279800709968334,habitación en gavà,46.0,4.79,95,barcelona,españa,habitación,https://www.airbnb.com.pe/rooms/99427980070996...
266,1192612229323894350,apartamento en barcelona,91.0,0.00,0,barcelona,españa,apartamento,https://www.airbnb.com.pe/rooms/11926122293238...
267,1125955652813692522,habitación en barcelona,49.0,4.50,16,barcelona,españa,habitación,https://www.airbnb.com.pe/rooms/11259556528136...
268,908155559326571593,habitación en barcelona,46.0,4.75,4,barcelona,españa,habitación,https://www.airbnb.com.pe/rooms/90815555932657...
